In [1]:
import json
import re

import pandas as pd
from gensim.models.word2vec import Word2Vec

In [2]:
df = pd.read_csv("classification_articles.csv")

# Data preprocessing

In [3]:
# https://stackoverflow.com/questions/17796446/convert-a-list-to-a-string-and-back
df["sentences"] = df["sentences"].apply(lambda s: eval(s))
df["keyword_top10"] = df["keyword_top10"].apply(lambda s: eval(s))
df["locations"] = df["locations"].apply(lambda s: eval(s))
df["entity_address"] = df["entity_address"].apply(lambda s: eval(s))

In [4]:
df_classified = df.loc[~df["self_defined_category"].isna()]

In [5]:
df.shape

(872, 7)

In [6]:
df_classified.shape

(200, 7)

# Explore categories and keywords

In [7]:
df_classified["self_defined_category"].value_counts()

其它      79
自然景觀    51
人文藝術    38
娛樂購物    32
Name: self_defined_category, dtype: int64

In [8]:
other_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "其它", "sentences"]:
    other_sentences.extend(l)

In [9]:
series_other = pd.Series(other_sentences)
series_other.value_counts().head(10)

飯店    14
房間    12
空間    11
景點    10
推薦    10
住宿     9
台中     8
美食     8
設施     7
早餐     7
dtype: int64

In [10]:
series_other.value_counts()[series_other.value_counts() > 1]

飯店    14
房間    12
空間    11
景點    10
推薦    10
      ..
龍潭     2
嘉義     2
寶來     2
台北     2
時間     2
Length: 99, dtype: int64

In [11]:
nature_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "自然景觀", "sentences"]:
    nature_sentences.extend(l)

In [12]:
series_nature = pd.Series(nature_sentences)
series_nature.value_counts().head(10)

景點        16
櫻花        10
2019       9
js         9
露營         8
步道         8
window     7
push       7
花況         6
賞櫻         5
dtype: int64

In [13]:
series_nature.value_counts()[series_nature.value_counts() > 1]

景點      16
櫻花      10
2019     9
js       9
露營       8
        ..
農場       2
富士櫻      2
精油       2
小木屋      2
免費       2
Length: 64, dtype: int64

In [14]:
art_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "人文藝術", "sentences"]:
    art_sentences.extend(l)

In [15]:
series_art = pd.Series(art_sentences)
series_art.value_counts().head(10)

景點     11
宜蘭      5
園區      5
台灣      5
小朋友     4
ig      4
空間      4
體驗      4
建築      4
打卡      4
dtype: int64

In [16]:
series_art.value_counts()[series_art.value_counts() > 1]

景點        11
宜蘭         5
園區         5
台灣         5
小朋友        4
ig         4
空間         4
體驗         4
建築         4
打卡         4
藝術         3
台中         3
博物館        3
武淵         3
推薦         3
公園         3
免費         3
水火同源       3
閱讀         3
現代         2
日式         2
diy        2
參觀         2
客家         2
苗栗市        2
地景         2
書店         2
桃園         2
大碗公溜滑梯     2
門票         2
沙雕         2
陀螺         2
龍潭         2
火車         2
草皮         2
音樂         2
溜滑梯        2
鍾肇政        2
協櫻         2
宿舍         2
戲水         2
餐廳         2
機芯         2
展覽         2
創作         2
泡腳         2
青鳥         2
幾米         2
華山         2
咖啡         2
台南         2
指尖         2
時間         2
活動         2
日式建築       2
dtype: int64

In [17]:
shop_sentences = []
for l in df_classified.loc[df_classified["self_defined_category"] == "娛樂購物", "sentences"]:
    shop_sentences.extend(l)

In [18]:
series_shop = pd.Series(shop_sentences)
series_shop.value_counts().head(10)

小朋友    6
溜滑梯    5
js     4
公園     4
台中     3
特賣會    3
宜蘭     3
台南     3
diy    3
中壢     3
dtype: int64

In [19]:
series_shop.value_counts()[series_shop.value_counts() > 1]

小朋友       6
溜滑梯       5
js        4
公園        4
台中        3
特賣會       3
宜蘭        3
台南        3
diy       3
中壢        3
小米        3
桃園        3
打卡        3
可愛        2
觀光工廠      2
開幕        2
大江購物中心    2
體驗        2
台中后里      2
門票        2
喜歡        2
空間        2
參考        2
球池        2
吉米        2
設施        2
美食        2
push      2
id        2
window    2
花蓮        2
台灣        2
遊戲場       2
親子        2
dtype: int64

In [20]:
# Find intersections of top-10-common keywords accross categories
# (to be excluded in following predictions)
series_common = (
    series_other.value_counts().head(10).index
        .append(series_art.value_counts().head(10).index)
        .append(series_nature.value_counts().head(10).index)
        .append(series_shop.value_counts().head(10).index)
)
series_common.value_counts()

景點        3
台中        2
空間        2
小朋友       2
js        2
宜蘭        2
diy       1
建築        1
櫻花        1
中壢        1
推薦        1
push      1
設施        1
特賣會       1
步道        1
住宿        1
體驗        1
ig        1
早餐        1
飯店        1
溜滑梯       1
打卡        1
公園        1
園區        1
賞櫻        1
美食        1
台灣        1
2019      1
花況        1
window    1
露營        1
房間        1
台南        1
dtype: int64

In [21]:
list_excluding_keywords = series_common.value_counts()[series_common.value_counts() > 1].index.tolist()
list_excluding_keywords

['景點', '台中', '空間', '小朋友', 'js', '宜蘭']

In [22]:
# Keywords can't be locations either
list_excluding_locations = [s for l in df_classified["locations"] for s in l]
list_excluding_locations = list(set(list_excluding_locations))
list_excluding_locations

['東豐自行車綠廊',
 '新北',
 '高雄',
 '太原',
 '在水一方',
 '彩虹',
 '梅花',
 '臺中市眷村文物館',
 '勤美綠園道',
 '花蓮',
 '桃園國際棒球場',
 '綠園道',
 '美術館',
 '鰲峰山運動公園',
 '士林',
 '青埔',
 '水舞行館',
 '中山',
 '宜蘭伯朗大道',
 '嘉義',
 '谷關',
 '準園休閒農場',
 '苗栗',
 '卓蘭',
 '北投',
 '二崁聚落',
 '曼谷',
 '綠川',
 '保安車站',
 '童話世界',
 '東大門夜市',
 '武荖坑風景區',
 '陽明山天籟渡假酒店',
 '勤美術館',
 '峇里島',
 '板橋',
 '冬山',
 '三重',
 '赤崁璽樓民宿',
 '楠西',
 '大雅',
 '南寮',
 '七股',
 '西屯',
 '恆春',
 '太平山森林遊樂區',
 '冬山鄉',
 '綠舞國際觀光飯店',
 '后里區',
 '鯉魚潭',
 '村却國際溫泉酒店',
 '樹谷農場',
 '華山',
 '花蓮港',
 '遠雄海洋公園',
 '大安區',
 '歐洲',
 '望龍埤',
 '環保公園',
 '亞太飯店',
 '華山文創園區',
 '青草湖',
 '大溪老街',
 '紅樹林站',
 '運動公園',
 '大陸',
 '壽豐',
 '台江國家公園遊客中心',
 '小琉球',
 '新化',
 '墾丁',
 '文物館',
 '玉井',
 '海南',
 '拉拉山恩愛農場',
 '總爺藝文中心',
 '清水',
 '花蓮理想<ModifierP>大地</ModifierP>渡假飯店',
 '南方莊園渡假飯店',
 '松山區',
 '休閒農業區',
 '南京三民站',
 '蘆竹',
 '泰安',
 '巴黎',
 '月眉糖廠',
 '功維敘隧道',
 '室內',
 '達邦',
 '觀光夜市',
 '月亮',
 '新竹',
 '台北車站',
 '南印度',
 '六龜區',
 '天元宮',
 '台南',
 '大甲溪',
 '桃園',
 '台灣',
 '露營區',
 '報告班長',
 '大溪',
 '鳥人創意旅店',
 '白河',
 '美國',
 '中壢',
 '東港',
 '世界',
 '展覽館',
 '印度',
 '羅東',
 '黃金瀑布',
 '菱潭街興創基地',
 

In [23]:
df["sentences"] = df["sentences"].apply(
    lambda l: [
        s for s in l
        # Excluding common keywords
        if s not in list_excluding_keywords and
        # Excluding locations
        s not in list_excluding_locations and
        # Excluding pure digits
        not re.match(r"\d+$", s)
    ]
)

# Word2Vec

In [24]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except KeyError:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [25]:
model = Word2Vec(df["sentences"], size=250, iter=10, min_count=2, sg=1, window=1)

In [26]:
most_similar(model, ["櫻花", "溜滑梯", "門票", "飯店", "民宿", "步道", "芒果"])

,櫻花,cos,溜滑梯,cos,門票,cos,飯店,cos,民宿,cos,步道,cos,芒果,cos
0,體驗,0.608565,飯店,0.759139,活動,0.701211,溜滑梯,0.759139,房間,0.731666,入住,0.748033,溜滑梯,0.446308
1,飯店,0.605470,餐廳,0.729688,房間,0.701100,體驗,0.748625,參觀,0.722958,飯店,0.742038,感覺,0.436269
2,民宿,0.599264,房間,0.714571,打卡,0.694466,房間,0.743905,感覺,0.718320,打卡,0.738534,適合,0.430391
3,房間,0.598699,免費,0.710693,步道,0.692031,步道,0.742038,飯店,0.717013,房間,0.732990,公園,0.422715
4,diy,0.598142,喜歡,0.697841,飯店,0.689467,喜歡,0.738174,設施,0.712790,喜歡,0.714674,diy,0.419838
5,秘境,0.589898,體驗,0.685985,喜歡,0.689136,餐廳,0.736548,體驗,0.712573,餐廳,0.713339,賞花,0.416683
6,感覺,0.581717,賞花,0.684113,入住,0.688856,感覺,0.734680,步道,0.711689,民宿,0.711689,民宿,0.416543
7,參觀,0.578503,民宿,0.683397,餐廳,0.686794,免費,0.727957,喜歡,0.700955,體驗,0.701031,藝術,0.411484
8,口感,0.574654,美食,0.677411,體驗,0.678083,參觀,0.724719,園區,0.685909,diy,0.700090,觀光工廠,0.409977
9,入住,0.571422,diy,0.676630,親子,0.669407,入住,0.718494,溜滑梯,0.683397,感覺,0.699552,步道,0.404663


In [27]:
dict_df_classified = df.where(df.notnull(), None).to_dict()
with open("preprocessed_data.json", "w") as f:
    json.dump(dict_df_classified, f, indent=2, ensure_ascii=False)

In [28]:
model.save("word2vec.model")